In [47]:
import numpy as np
import numpy.ma as ma
import rasterio
from affine import Affine
from sklearn.decomposition import PCA

## 1a. Let's prepare a 3-band raster

In [48]:
bands = 4
rows = 5
cols = 6
num = bands * rows * cols

In [49]:
data = np.linspace(1, num, num)
# data = np.random.rand(bands, rows, cols)

In [50]:
data

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100., 101., 102., 103., 104., 105., 106., 107., 108., 109., 110.,
       111., 112., 113., 114., 115., 116., 117., 118., 119., 120.])

In [51]:
data = data.reshape(bands, rows, cols)

In [52]:
data

array([[[  1.,   2.,   3.,   4.,   5.,   6.],
        [  7.,   8.,   9.,  10.,  11.,  12.],
        [ 13.,  14.,  15.,  16.,  17.,  18.],
        [ 19.,  20.,  21.,  22.,  23.,  24.],
        [ 25.,  26.,  27.,  28.,  29.,  30.]],

       [[ 31.,  32.,  33.,  34.,  35.,  36.],
        [ 37.,  38.,  39.,  40.,  41.,  42.],
        [ 43.,  44.,  45.,  46.,  47.,  48.],
        [ 49.,  50.,  51.,  52.,  53.,  54.],
        [ 55.,  56.,  57.,  58.,  59.,  60.]],

       [[ 61.,  62.,  63.,  64.,  65.,  66.],
        [ 67.,  68.,  69.,  70.,  71.,  72.],
        [ 73.,  74.,  75.,  76.,  77.,  78.],
        [ 79.,  80.,  81.,  82.,  83.,  84.],
        [ 85.,  86.,  87.,  88.,  89.,  90.]],

       [[ 91.,  92.,  93.,  94.,  95.,  96.],
        [ 97.,  98.,  99., 100., 101., 102.],
        [103., 104., 105., 106., 107., 108.],
        [109., 110., 111., 112., 113., 114.],
        [115., 116., 117., 118., 119., 120.]]])

In [53]:
data.shape

(4, 5, 6)

In [54]:
print(np.amin(data))
print(np.amax(data))

1.0
120.0


## 1b. Now let's make the surrounding pixels be nan's...no data values for the skewed part of an image

In [55]:
print(data.shape[0])
print(data.shape[1])
print(data.shape[2])

4
5
6


In [56]:
data[:, :, 0] = np.nan # first column
data[:, 0, :] = np.nan # first row
data[:, :, data.shape[2]-1] = np.nan # last column
data[:, data.shape[1]-1, :] = np.nan # last row
data[:, 1, 1] = np.nan # upper-left corner
data[:, data.shape[1]-2, data.shape[2]-2] = np.nan # lower-right corner

In [57]:
data

array([[[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,   9.,  10.,  11.,  nan],
        [ nan,  14.,  15.,  16.,  17.,  nan],
        [ nan,  20.,  21.,  22.,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  39.,  40.,  41.,  nan],
        [ nan,  44.,  45.,  46.,  47.,  nan],
        [ nan,  50.,  51.,  52.,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  69.,  70.,  71.,  nan],
        [ nan,  74.,  75.,  76.,  77.,  nan],
        [ nan,  80.,  81.,  82.,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  99., 100., 101.,  nan],
        [ nan, 104., 105., 106., 107.,  nan],
        [ nan, 110., 111., 112.,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]]])

In [58]:
data.shape

(4, 5, 6)

In [59]:
data.ndim

3

In [60]:
np.nanmin(data)

9.0

In [61]:
np.nanmax(data)

112.0

# __2. Getting to work: trimming out the nan's and mapping them back__

### TRIMMING, version 1...
    ...where the masks of all bands are the same (perhaps an unrealistic situation?)

In [35]:
mask = np.isnan(data)
print(mask)
print(mask.shape)

[[[ True  True  True  True  True  True]
  [ True  True False False False  True]
  [ True False False False False  True]
  [ True False False False  True  True]
  [ True  True  True  True  True  True]]

 [[ True  True  True  True  True  True]
  [ True  True False False False  True]
  [ True False False False False  True]
  [ True False False False  True  True]
  [ True  True  True  True  True  True]]

 [[ True  True  True  True  True  True]
  [ True  True False False False  True]
  [ True False False False False  True]
  [ True False False False  True  True]
  [ True  True  True  True  True  True]]

 [[ True  True  True  True  True  True]
  [ True  True False False False  True]
  [ True False False False False  True]
  [ True False False False  True  True]
  [ True  True  True  True  True  True]]]
(4, 5, 6)


In [36]:
trimmed = data[~mask]

In [37]:
trimmed

array([  9.,  10.,  11.,  14.,  15.,  16.,  17.,  20.,  21.,  22.,  39.,
        40.,  41.,  44.,  45.,  46.,  47.,  50.,  51.,  52.,  69.,  70.,
        71.,  74.,  75.,  76.,  77.,  80.,  81.,  82.,  99., 100., 101.,
       104., 105., 106., 107., 110., 111., 112.])

In [38]:
trimmed.shape

(40,)

In [39]:
trimmed.ndim

1

In [40]:
trimmed = trimmed.reshape(bands, int(len(trimmed)/bands)).T
trimmed

array([[  9.,  39.,  69.,  99.],
       [ 10.,  40.,  70., 100.],
       [ 11.,  41.,  71., 101.],
       [ 14.,  44.,  74., 104.],
       [ 15.,  45.,  75., 105.],
       [ 16.,  46.,  76., 106.],
       [ 17.,  47.,  77., 107.],
       [ 20.,  50.,  80., 110.],
       [ 21.,  51.,  81., 111.],
       [ 22.,  52.,  82., 112.]])

In [41]:
trimmed.shape

(10, 4)

In [42]:
print(np.min(trimmed))
print(np.max(trimmed))
print(np.mean(trimmed))

9.0
112.0
60.5


### TRIMMING, version 1...AS A FUNCTION
    ...where the masks of all bands are the same (perhaps an unrealistic situation?)

In [43]:
def trim_raster(data):
    """
    Parameters
    ----------
    data:
        A Rasterio-style numpy array (i.e., bands, rows, columns)
    
    Returns
    ------
    (1) the trimmed data...no nan values
    (2) the masked data array in Rasterio-style
    """
    
    bands, rows, cols = data.shape
    mask = np.isnan(data)
    trimmed = data[~mask]
    trimmed = trimmed.reshape(bands, int(len(trimmed)/bands)).T
    return trimmed, np.invert(mask)

In [ ]:
def trim_raster2(data):
    """
    Parameters
    ----------
    data:
        A Rasterio-style numpy array (i.e., bands, rows, columns)
    
    Returns
    ------
    (1) the trimmed data...no nan values
    (2) the masked data array in Rasterio-style
    """
    
    bands, rows, cols = data.shape
    mask = np.isnan(data)
    trimmed = data[~mask]
    trimmed = trimmed.reshape(bands, int(len(trimmed)/bands)).T
    return trimmed, np.invert(mask)

In [44]:
trimmed2, mask2 = trim_raster(data)
print(trimmed2)
print("\n\n")
print(mask2)

[[  9.  39.  69.  99.]
 [ 10.  40.  70. 100.]
 [ 11.  41.  71. 101.]
 [ 14.  44.  74. 104.]
 [ 15.  45.  75. 105.]
 [ 16.  46.  76. 106.]
 [ 17.  47.  77. 107.]
 [ 20.  50.  80. 110.]
 [ 21.  51.  81. 111.]
 [ 22.  52.  82. 112.]]



[[[False False False False False False]
  [False False  True  True  True False]
  [False  True  True  True  True False]
  [False  True  True  True False False]
  [False False False False False False]]

 [[False False False False False False]
  [False False  True  True  True False]
  [False  True  True  True  True False]
  [False  True  True  True False False]
  [False False False False False False]]

 [[False False False False False False]
  [False False  True  True  True False]
  [False  True  True  True  True False]
  [False  True  True  True False False]
  [False False False False False False]]

 [[False False False False False False]
  [False False  True  True  True False]
  [False  True  True  True  True False]
  [False  True  True  True False False]


In [46]:
print(data.shape)
print(trimmed2.shape)
print(mask2.shape)

(4, 5, 6)
(10, 4)
(4, 5, 6)


### MAPPING BACK, version 1...
    ...where the masks of all bands are the same (perhaps an unrealistic situation?)

In [27]:
bands, rows, cols = mask2.shape
output = np.empty((bands, rows, cols))
output[:, :, :] = np.nan
for b in range(bands):
    id = 0
    for r in range(rows):
        for c in range(cols):
            if mask2[b, r, c]==True:
                output[b, r, c] = trimmed[id, b]
                id+=1
output

array([[[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,   9.,  10.,  11.,  nan],
        [ nan,  14.,  15.,  16.,  17.,  nan],
        [ nan,  20.,  21.,  22.,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  39.,  40.,  41.,  nan],
        [ nan,  44.,  45.,  46.,  47.,  nan],
        [ nan,  50.,  51.,  52.,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  69.,  70.,  71.,  nan],
        [ nan,  74.,  75.,  76.,  77.,  nan],
        [ nan,  80.,  81.,  82.,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  99., 100., 101.,  nan],
        [ nan, 104., 105., 106., 107.,  nan],
        [ nan, 110., 111., 112.,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]]])

### MAPPING BACK, version 1...AS A FUNCTION
    ...where the masks of all bands are the same (perhaps an unrealistic situation?)

In [72]:
def mapback_raster(data, mask):
    bands, rows, cols = mask.shape
    output = np.empty((bands, rows, cols))
    output[:, :, :] = np.nan
    for b in range(bands):
        id = 0
        for r in range(rows):
            for c in range(cols):
                if mask[b, r, c]==True:
                    output[b, r, c] = data[id, b]
                    id+=1
    
    return output

In [73]:
mapback_raster(trimmed2, mask2)

array([[[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,   9.,  10.,  11.,  nan],
        [ nan,  14.,  15.,  16.,  17.,  nan],
        [ nan,  20.,  21.,  22.,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  39.,  40.,  41.,  nan],
        [ nan,  44.,  45.,  46.,  47.,  nan],
        [ nan,  50.,  51.,  52.,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  69.,  70.,  71.,  nan],
        [ nan,  74.,  75.,  76.,  77.,  nan],
        [ nan,  80.,  81.,  82.,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  99., 100., 101.,  nan],
        [ nan, 104., 105., 106., 107.,  nan],
        [ nan, 110., 111., 112.,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]]])

# __3. Now repeat with a twist--different nan's in different bands__

### 3.a altering the data

In [31]:
dat = data
dat[3, 3, 3] = np.nan # This replaces the highest value pixel with nan
dat

array([[[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,   9.,  10.,  11.,  nan],
        [ nan,  14.,  15.,  16.,  17.,  nan],
        [ nan,  20.,  21.,  22.,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  39.,  40.,  41.,  nan],
        [ nan,  44.,  45.,  46.,  47.,  nan],
        [ nan,  50.,  51.,  52.,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  69.,  70.,  71.,  nan],
        [ nan,  74.,  75.,  76.,  77.,  nan],
        [ nan,  80.,  81.,  82.,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  99., 100., 101.,  nan],
        [ nan, 104., 105., 106., 107.,  nan],
        [ nan, 110., 111.,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]]])

### TRIMMING, version 2
    ...with (potentially) different masks for each band (probably a realistic situation)

In [32]:
msk = np.isnan(dat)
msk

array([[[ True,  True,  True,  True,  True,  True],
        [ True,  True, False, False, False,  True],
        [ True, False, False, False, False,  True],
        [ True, False, False, False,  True,  True],
        [ True,  True,  True,  True,  True,  True]],

       [[ True,  True,  True,  True,  True,  True],
        [ True,  True, False, False, False,  True],
        [ True, False, False, False, False,  True],
        [ True, False, False, False,  True,  True],
        [ True,  True,  True,  True,  True,  True]],

       [[ True,  True,  True,  True,  True,  True],
        [ True,  True, False, False, False,  True],
        [ True, False, False, False, False,  True],
        [ True, False, False, False,  True,  True],
        [ True,  True,  True,  True,  True,  True]],

       [[ True,  True,  True,  True,  True,  True],
        [ True,  True, False, False, False,  True],
        [ True, False, False, False, False,  True],
        [ True, False, False,  True,  True,  True],
      

In [33]:
msk = np.any(msk, axis=0)
msk

array([[ True,  True,  True,  True,  True,  True],
       [ True,  True, False, False, False,  True],
       [ True, False, False, False, False,  True],
       [ True, False, False,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True]])

In [34]:
trmmd = dat[:, ~msk]

In [35]:
trmmd = trmmd.T

In [37]:
trmmd

array([[  9.,  39.,  69.,  99.],
       [ 10.,  40.,  70., 100.],
       [ 11.,  41.,  71., 101.],
       [ 14.,  44.,  74., 104.],
       [ 15.,  45.,  75., 105.],
       [ 16.,  46.,  76., 106.],
       [ 17.,  47.,  77., 107.],
       [ 20.,  50.,  80., 110.],
       [ 21.,  51.,  81., 111.]])

In [38]:
trmmd.shape

(9, 4)

In [39]:
trmmd.ndim

2

In [40]:
print(np.min(trmmd))
print(np.max(trmmd))
print(np.mean(trmmd))

9.0
111.0
59.77777777777778


### TRIMMING, version 2...AS A FUNCTION
    ...with (potentially) different masks for each band (probably a realistic situation)

In [41]:
def trim_raster2(data):
    """
    Parameters
    ----------
    data:
        A Rasterio-style numpy array (i.e., bands, rows, columns)
    
    Returns
    ------
    (1) the trimmed data...no nan values
    (2) the masked data array in Rasterio-style
    """
    
    bands, rows, cols = data.shape
    mask = np.isnan(data)
    mask = np.any(mask, axis=0) 
    trimmed = data[:, ~mask].T
    return trimmed, np.invert(mask)

In [42]:
trmmd2, msk2 = trim_raster2(dat)
print(trmmd2)
print("\n\n")
print(msk2)

[[  9.  39.  69.  99.]
 [ 10.  40.  70. 100.]
 [ 11.  41.  71. 101.]
 [ 14.  44.  74. 104.]
 [ 15.  45.  75. 105.]
 [ 16.  46.  76. 106.]
 [ 17.  47.  77. 107.]
 [ 20.  50.  80. 110.]
 [ 21.  51.  81. 111.]]



[[False False False False False False]
 [False False  True  True  True False]
 [False  True  True  True  True False]
 [False  True  True False False False]
 [False False False False False False]]


### MAPPING BACK, version 2...
    ...with (potentially) different masks for each band (probably a realistic situation)

In [43]:
dat.shape

(4, 5, 6)

In [44]:
bands, rows, cols = dat.shape
output = np.empty((bands, rows, cols))
output[:, :, :] = np.nan
for b in range(bands):
    id = 0
    for r in range(rows):
        for c in range(cols):
            if msk2[r, c]==True:
                output[b, r, c] = trmmd2[id, b]
                id+=1
output

array([[[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,   9.,  10.,  11.,  nan],
        [ nan,  14.,  15.,  16.,  17.,  nan],
        [ nan,  20.,  21.,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  39.,  40.,  41.,  nan],
        [ nan,  44.,  45.,  46.,  47.,  nan],
        [ nan,  50.,  51.,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  69.,  70.,  71.,  nan],
        [ nan,  74.,  75.,  76.,  77.,  nan],
        [ nan,  80.,  81.,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  99., 100., 101.,  nan],
        [ nan, 104., 105., 106., 107.,  nan],
        [ nan, 110., 111.,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]]])

### MAPPING BACK, version 2...AS A FUNCTION
    ...with (potentially) different masks for each band (probably a realistic situation)

In [49]:
def mapback_raster2(data, mask, bands):
    rows, cols = mask.shape
    output = np.empty((bands, rows, cols))
    output[:, :, :] = np.nan
    for b in range(bands):
        id = 0
        for r in range(rows):
            for c in range(cols):
                if mask[r, c]==True:
                    output[b, r, c] = data[id, b]
                    id+=1
    
    return output

In [50]:
mapback_raster2(trmmd2, msk2, dat.shape[0])

array([[[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,   9.,  10.,  11.,  nan],
        [ nan,  14.,  15.,  16.,  17.,  nan],
        [ nan,  20.,  21.,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  39.,  40.,  41.,  nan],
        [ nan,  44.,  45.,  46.,  47.,  nan],
        [ nan,  50.,  51.,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  69.,  70.,  71.,  nan],
        [ nan,  74.,  75.,  76.,  77.,  nan],
        [ nan,  80.,  81.,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]],

       [[ nan,  nan,  nan,  nan,  nan,  nan],
        [ nan,  nan,  99., 100., 101.,  nan],
        [ nan, 104., 105., 106., 107.,  nan],
        [ nan, 110., 111.,  nan,  nan,  nan],
        [ nan,  nan,  nan,  nan,  nan,  nan]]])

## OK. Let's try PCA

In [51]:
pca = PCA(n_components=4)

In [52]:
pca.fit(trmmd2)

PCA(copy=True, iterated_power='auto', n_components=4, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [53]:
print(pca.explained_variance_ratio_) 

[1.00000000e+00 8.37381552e-31 6.64056568e-34 4.87678248e-63]
